In [ ]:
from IPython.core.display import HTML
with open('style.css') as file:
    css = file.read()
HTML(css)

In [ ]:
# Autload python modules by default
%load_ext autoreload
%autoreload 2

# Convert notebooks to python, so they can be loaded effiently
from utils.jupyter_loader import JupyterLoader

loader = JupyterLoader()
loader.load_all()

# Move Ordering

Before doing any further optimization, let's first check the current performance of the latest engine. For the middlegame position the `PrototypeV1Engine` engine visits `25139` nodes and needs approximatly 5 seconds on our machine. 

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator
from converted_notebooks.s14_prototype_v1 import PrototypeV1Engine

random.seed(42)

prototype_v1_engine = PrototypeV1Engine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)

tree = utils.min_max_tree.add_tree_to_engine(prototype_v1_engine)
prototype_v1_engine.play(middlegame_board)
print(tree.count())

# %timeit prototype_v1_engine.play(middlegame_board)

One of the key reasons to do iterative deepening in the first place, is to make use of the results from previous iterations. Provided a good evaluation function is used, a reasonable assumption is that the evaluation of each move won't change much if it's analysed one depth further. This information can be used to sort moves in the next iteration. Alpha Beta prunning leads to the best results if the best move is searched for, therefore we sort the moves according to their score. 

The `_value` method will use a helper function `_get_moves` to get a sorted list of moves rather than `board.legal_moves`. The helper method takes the current board `board` and the depth `depth` as parameter and returns the list of all possible next moves, which are sorted by their value in the previous iteration. 

In [ ]:
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta
from converted_notebooks.s12_simplified_evaluation import DetailedMove
from converted_notebooks.s14_prototype_v1 import PrototypeV1Engine
import chess
from chess.engine import PovScore


class MoveOrderingEngine(PrototypeV1Engine):

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: int, beta: int
    ) -> PovScore:
        if (score := self.evaluator.evaluate(board)) is not None:
            return score
        if depth == 0:
            return self._quiescence(board, alpha, beta)

        for move in self._get_moves(board, depth):
            detailedMove = DetailedMove(board, move)

            self.evaluator.push(detailedMove)
            board.push(move)
            value = self._value(board, depth - 1, -beta,
                                -alpha).pov(not board.turn)
            board.pop()
            self.evaluator.pop()

            if value >= beta:
                return PovScore(value, board.turn)
            alpha = max(alpha, value)

        return PovScore(alpha, board.turn)

    def _get_moves(self, board: chess.Board, depth: int):
        cached_moves = []
        uncached_moves = []
        for move in board.legal_moves:
            board.push(move)
            cache_key = self.cache.get_key(board)
            board.pop()

            try:
                # depth - 2 as we are looking at one move further and at the previous iteration with depth - 1
                _, value = self.cache.load_cache(cache_key, depth - 2)
                cached_moves.append((value.relative, move))
            except KeyError:
                uncached_moves.append(move)

        cached_moves.sort(reverse=False, key=lambda x: x[0])
        return [cached_move[1] for cached_move in cached_moves] + uncached_moves

If we ran the same analyse as before with the new defined engine, then the number of nodes drasticall decreases from `25139` to `12911`. Unfortunatly, due to the overhead of the added code, the time decreased only by approximatly one second from `5` seconds to `4` seconds on our machine. Nevertheless, this is still a huge improvement.

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board

random.seed(42)

# IPython.display.display(middlegame_board)

move_ordering_engine = MoveOrderingEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=4
)

# tree = utils.min_max_tree.add_tree_to_engine(move_ordering_engine)
# move_ordering_engine.play(middlegame_board)
# print(tree.count())

%timeit move_ordering_engine.play(middlegame_board)

Sorting every move according to their score implies that the principal variation, therefore best line in the previous iteration, is searched first. Furthermore, at Cut Nodes the move that lead to the cut in the previous iteration, also called refutation move, is searched first as well. These are the two most important move criteria according to [the chessprogramming wiki](https://www.chessprogramming.org/Move_Ordering#Typical_move_ordering). The only downside of the current approach is that the best move or refutation move is not already stored in the transposition table. Changing this wouldn't improve the number of visited nodes, but the time needed to visit one node.

Next, another look should be taken on the `quiescence search`. We can calculate how many nodes are visited in the quiescence search. With `513446` the number of nodes spent in the quiescence search is by factor 50 higher than in the normal alpha beta search. This doesn't mean though that it is not worth it to optimize the `_value` function, quite the opposite is true. Every node visited less in the `_value` function will lead to a huge decrease of nodes in the `quiescence` search. Nonetheless, it might be still useful to try to optimize `_quiescence` as well.

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board

random.seed(42)

move_ordering_engine = MoveOrderingEngine(
    evaluator=incremental_simplified_evaluator, max_look_ahead_depth=5
)

tree = utils.min_max_tree.add_tree_to_engine(move_ordering_engine)
move_ordering_engine.play(middlegame_board)
print(tree.count(quiesce=True))

# %timeit move_ordering_engine.play(middlegame_board)

# 3.91 s ± 67.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# 12911 nodes visited
# 151349 paths pruned
# 513446 quiesce nodes visited
# 333054 quiesce paths pruned

We define a new class `QuiescenceMoveOrderingEngine` which inherits from `MoveOrderingEngine`. The `_quiescence` uses a helper method `_get_quiescence_detailed_moves` to get all moves that should be checked in sorted order. The optimization like delta pruning or static exchange evaluation are moved into that function as well. The new `_get_quiescence_detailed_moves` method takes the board `board`, the standing pat `stand_path` and the alpha and beta value `alpha` and `beta` as parameter and returns a sorted list of next moves. As before, only captures are considered and some of these are pruned by delta pruning or static exchange evaluation. The moves are sorted by MVV-LVA, which stands fore most valuable victim - least valuable attacker. Therefore the moves are first sorted by the most valuable captured piece and in case of an qual captured then sorted by the least valuable attacker. As the value of the captured piece and moved piece have to be calculated anyway, the two methods `_seeCapture` and `_canDeltaPrune` have been rewritten to use this information for better performance.

In [ ]:
from converted_notebooks.s12_simplified_evaluation import IncrementalEvaluator
from chess.engine import Score, Cp, Mate, PovScore


class QuiescenceMoveOrderingEngine(MoveOrderingEngine):

    def __init__(
        self,
        evaluator: IncrementalEvaluator,
        max_look_ahead_depth,
        use_static_exchange_evaluation=False
    ):
        super().__init__(evaluator, max_look_ahead_depth)
        self.use_static_exchange_evaluation = use_static_exchange_evaluation

    def _quiescence(self, board: chess.Board, alpha: Score, beta: Score) -> int:
        stand_pat = self.evaluator.get_score().relative

        if stand_pat >= beta:
            return PovScore(beta, board.turn)
        alpha = max(alpha, stand_pat)

        for detailed_move in self._get_quiescence_detailed_moves(
            board, stand_pat, alpha, beta
        ):
            self.evaluator.push(detailed_move)
            board.push(detailed_move.move)
            value = self._quiescence(board, -beta, -alpha).pov(not board.turn)
            board.pop()
            self.evaluator.pop()

            if value.is_mate():
                value = self._rise_mate(value)

            if value >= beta:
                return PovScore(value, board.turn)
            alpha = max(alpha, value)
        return PovScore(alpha, board.turn)

    def _get_quiescence_detailed_moves(
        self, board: chess.Board, stand_pat: int, alpha: int, beta: int
    ):
        moves = []
        for move in board.generate_legal_captures():
            detailedMove = DetailedMove(board, move)

            capturedPieceValue = self.evaluator.piece_values[
                detailedMove.capturedPiece.piece.piece_type]

            movedPieceValue = self.evaluator.piece_values[
                detailedMove.movedPiece.piece.piece_type]

            if move.promotion is None:
                if self._canDeltaPrune(stand_pat, alpha, capturedPieceValue):
                    continue

                if self._bad_capture(
                    board, detailedMove, capturedPieceValue, movedPieceValue
                ):
                    continue

            moves.append((capturedPieceValue, movedPieceValue, detailedMove))

        # Oder by MVV-LVA (most valuable victim first, least valuable attacker second)
        moves.sort(reverse=True, key=lambda x: (x[0], -x[1]))

        return [move[2] for move in moves]

    def _bad_capture(
        self,
        board: chess.Board,
        detailedMove: DetailedMove,
        capturedPieceValue: int,
        movedPieceValue: int
    ):
        value = capturedPieceValue - movedPieceValue
        if value >= 0:
            return False

        if self.use_static_exchange_evaluation:
            return self._seeCapture(
                board, detailedMove, capturedPieceValue, movedPieceValue
            ) < 0
        return board.is_attacked_by(
            not board.turn, detailedMove.placedPiece.square
        )

    def _seeCapture(
        self,
        original_board: chess.Board,
        detailedMove: DetailedMove,
        capturedPieceValue: int,
        movedPieceValue: int
    ):
        board = original_board.copy()

        piece = board.remove_piece_at(detailedMove.movedPiece.square)
        value = capturedPieceValue - self._see(
            board,
            detailedMove.placedPiece.square,
            movedPieceValue,
            not board.turn
        )
        board.set_piece_at(detailedMove.movedPiece.square, piece)

        return value

    def _see(
        self,
        board: chess.Board,
        square: chess.Square,
        attacked_piece_value: int,
        turn: chess.Color
    ):
        attackers = [
            (square, self.evaluator.piece_values[board.piece_type_at(square)])
            for square in board.attackers(turn, square)
        ]
        if not attackers:
            return 0

        attacker_square, attacker_piece_value = min(attackers, key=lambda x: x[1])

        piece = board.remove_piece_at(attacker_square)
        value = max(
            0,
            attacked_piece_value -
            self._see(board, square, attacker_piece_value, not turn)
        )
        board.set_piece_at(attacker_square, piece)
        return value

    def _canDeltaPrune(
        self, stand_pat: Score, alpha: Score, capturedPieceValue: int
    ):
        POTENTIAL_POSITION_ADVANTAGE = 200
        bestAlpha = Cp(
            stand_pat.score() + capturedPieceValue +
            POTENTIAL_POSITION_ADVANTAGE
        )
        return bestAlpha < alpha

A test shows that the number of visited nodes is decreased from `513446` to `467973`, so approximatly 10% of the nodes are saved. 

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

quiescence_move_ordering_engine = QuiescenceMoveOrderingEngine(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=False
)

# tree = utils.min_max_tree.add_tree_to_engine(quiescence_move_ordering_engine)
# quiescence_move_ordering_engine.play(middlegame_board)
# print(tree.count(quiesce=True))

%timeit quiescence_move_ordering_engine.play(middlegame_board)

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

quiescence_move_ordering_engine = QuiescenceMoveOrderingEngine(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=True
)

# tree = utils.min_max_tree.add_tree_to_engine(quiescence_move_ordering_engine)
# quiescence_move_ordering_engine.play(middlegame_board)
# print(tree.count(quiesce=True))

# %timeit quiescence_move_ordering_engine.play(middlegame_board)

results = quiescence_move_ordering_engine.analyse(middlegame_board)

# Tree Statistics
# 12911 nodes visited
# 151455 paths pruned
# 452292 quiesce nodes visited
# 273771 quiesce paths pruned

# Tree Statistics
# 3427 nodes visited
# 87240 paths pruned
# 283597 quiesce nodes visited
# 173305 quiesce paths pruned

# 3.8 s ± 86.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)

# Principal Variation Search

In [ ]:
from converted_notebooks.s11_iterative_deepening import cache_alpha_beta, NodeType
from converted_notebooks.s12_simplified_evaluation import DetailedMove
from converted_notebooks.s14_prototype_v1 import PrototypeV1Engine
import chess


class PrincipalVariationSearch(QuiescenceMoveOrderingEngine):

    @cache_alpha_beta
    def _value(
        self, board: chess.Board, depth: int, alpha: Score, beta: Score
    ) -> PovScore:
        if (score := self.evaluator.evaluate(board)) is not None:
            return score
        if depth == 0:
            return self._quiescence(board, alpha, beta)

        search_principal_variation = True
        for move in self._get_moves(board, depth):
            detailedMove = DetailedMove(board, move)

            self.evaluator.push(detailedMove)
            board.push(move)

            if search_principal_variation:
                value = self._value(board, depth - 1, -beta,
                                    -alpha).pov(not board.turn)
            else:
                value = self._zero_width_search(board, depth - 1,
                                                -alpha).pov(not board.turn)
                if alpha < value < beta:
                    value = self._value(board, depth - 1, -beta,
                                        -alpha).pov(not board.turn)

            board.pop()
            self.evaluator.pop()

            if value.is_mate():
                value = self._rise_mate(value)

            if value >= beta:
                return PovScore(value, board.turn)

            if value > alpha:
                alpha = value
                search_principal_variation = False

        return PovScore(alpha, board.turn)

    def _zero_width_search(
        self, board: chess.Board, depth: int, beta: Score
    ) -> PovScore:
        if beta.is_mate():
            alpha = Mate(beta.mate() +
                         1) if beta > Cp(0) else Mate(beta.mate() - 1)
        else:
            alpha = Cp(beta.score() - 1)

        if (score := self.evaluator.evaluate(board)) is not None:
            return score
        if depth == 0:
            return self._quiescence(board, alpha, beta)

        for move in board.generate_legal_moves():
            detailedMove = DetailedMove(board, move)

            self.evaluator.push(detailedMove)
            board.push(move)
            value = self._zero_width_search(board, depth - 1,
                                            -alpha).pov(not board.turn)
            board.pop()
            self.evaluator.pop()

            if value.is_mate():
                value = self._rise_mate(value)

            if value >= beta:
                return PovScore(value, board.turn)
        return PovScore(alpha, board.turn)

In [ ]:
import random
import utils.min_max_tree
import IPython.display
from converted_notebooks.s09_minimax_engine import middlegame_board
from converted_notebooks.s12_simplified_evaluation import incremental_simplified_evaluator

random.seed(42)

principal_variation_search_engine = PrincipalVariationSearch(
    evaluator=incremental_simplified_evaluator,
    max_look_ahead_depth=4,
    use_static_exchange_evaluation=False
)

# tree = utils.min_max_tree.add_tree_to_engine(principal_variation_search_engine)
# principal_variation_search_engine.play(middlegame_board)
# print(tree.count(quiesce=True))

%timeit principal_variation_search_engine.play(middlegame_board)

# Tree Statistics
# 3427 nodes visited
# 87240 paths pruned
# 283597 quiesce nodes visited
# 173305 quiesce paths pruned
# None

# 2.47 s ± 74.1 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) / SEE
# 1.4 s ± 18.4 ms per loop (mean ± std. dev. of 7 runs, 1 loop each) / without SEE